In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import tensorflow.keras.backend as K
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_pickle('train.pickle')
df.dropna(axis='columns', inplace=True)
df

In [ ]:
meter = df.meter
building_id = df.building_id
site_id = df.site_id
primary_use = df.primary_use
day = df.day - 1
hour = df.hour
weekday = df.weekday
numerics = df[['square_feet', 'air_temperature', 'dew_temperature', 'wind_direction', 'wind_speed', 'air_temperature_24', 'air_temperature_48', 'air_temperature_96', 'dew_temperature_24', 'dew_temperature_48', 'dew_temperature_96']]
numerics = StandardScaler().fit_transform(numerics)
meter_reading = df.meter_reading

In [ ]:
def generate_network():
    K.clear_session()
    
    meter = keras.layers.Input(1)
    building_id = keras.layers.Input(1)
    site_id = keras.layers.Input(1)
    primary_use = keras.layers.Input(1)
    day = keras.layers.Input(1)
    hour = keras.layers.Input(1)
    weekday = keras.layers.Input(1)
    numerics = keras.layers.Input(11)
    
    embeddings = []
    embeddings.append(keras.layers.Embedding(4, 32)(meter))
    embeddings.append(keras.layers.Embedding(1449, 32)(building_id))
    embeddings.append(keras.layers.Embedding(16, 32)(site_id))
    embeddings.append(keras.layers.Embedding(16, 32)(primary_use))
    embeddings.append(keras.layers.Embedding(31, 32)(day))
    embeddings.append(keras.layers.Embedding(24, 32)(hour))
    embeddings.append(keras.layers.Embedding(7, 32)(weekday))
    
    layers = [numerics]
    for embedding in embeddings:
        layers.append(keras.layers.Flatten()(embedding))        
    
    prev = keras.layers.Concatenate()(layers)
    
    residual = None
    for i in range(20):
        prev = keras.layers.Dense(64, kernel_initializer='he_uniform')(prev)
        prev = keras.layers.BatchNormalization(momentum=0.9)(prev)
        prev = keras.layers.Activation('elu')(prev)
        
        if residual is None:
            residual = prev
        else:
            prev = keras.layers.Add()([prev, residual])
            residual = prev
            
        prev = keras.layers.Dropout(0.5)(prev)
        
    outputs = keras.layers.Dense(1)(prev)
    
    return keras.models.Model(inputs=[meter, building_id, site_id, primary_use, day, hour, weekday, numerics], outputs=outputs)

In [ ]:
X = [meter, building_id, site_id, primary_use, day, hour, weekday, numerics]
y = meter_reading
earlystopping = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

model = generate_network()
model.compile(loss='mse')
model.fit(X, y, batch_size=32768, epochs=100, callbacks=[earlystopping], validation_split=0.25)